In [3]:
import urllib.request
url = ("https://raw.githubusercontent.com/rasbt/"
        "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
       "the-verdict.txt")
url_path = "the-verdict.txt"
urllib.request.urlretrieve(url,url_path)

('the-verdict.txt', <http.client.HTTPMessage at 0x10c04dfa0>)

In [1]:
with open('the-verdict.txt','r',) as f:
    raw_text = f.read()
    
print(len(raw_text))
print(raw_text[:99])

20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [2]:
import re
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(len(preprocessed))

4690


In [3]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)
print(vocab_size)

1130


In [4]:
vocab = {wrd:token for token,wrd in enumerate(all_words)}
for i,item in enumerate(vocab.items()):
    print(item)
    if i>50:
        break 

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)
('His', 51)


In [7]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
                                
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        preprocessed = [item if item in self.str_to_int
                        else "<|unk|>" for item in preprocessed]
        
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text

In [5]:
all_tokens = sorted(list(set(preprocessed))) 
all_tokens.extend(["<|endoftext|>", "<|unk|>"])
vocab = {token:integer for integer,token in enumerate(all_tokens)}                  

In [8]:
tokenizer = SimpleTokenizerV2(vocab)
text = "hello how are you ?"
print(tokenizer.encode(text))

[1131, 560, 169, 1126, 10]


In [9]:
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."
text = " <|endoftext|> ".join((text1, text2))
print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [10]:
tokenizer = SimpleTokenizerV2(vocab)
print(tokenizer.encode(text))

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]


In [11]:
print(tokenizer.decode(tokenizer.encode(text)))

<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.


In [12]:
from importlib.metadata import version
import tiktoken
print("tiktoken version:", version("tiktoken"))

tiktoken version: 0.8.0


In [13]:
tokenizer = tiktoken.get_encoding('gpt2')
text = (
            "Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunknownPlace."
)

integers = tokenizer.encode(text,allowed_special={"<|endoftext|>"})
integers

[15496,
 11,
 466,
 345,
 588,
 8887,
 30,
 220,
 50256,
 554,
 262,
 4252,
 18250,
 8812,
 2114,
 286,
 617,
 34680,
 27271,
 13]

In [14]:
string = tokenizer.decode(integers)
print(string)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunknownPlace.


In [15]:
with open("the-verdict.txt", "r", encoding="utf-8") as f: 
    raw_text = f.read()
enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


In [16]:
enc_sample = enc_text[50:]
context_size = 4
x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]
print(f'x: {x}')
print(f'y:      {y}')

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


In [17]:
for i in range(1,context_size+1):
    x = enc_sample[:i]
    y = enc_sample[i]
    print(f"input : {x} ----> output : {y}")

input : [290] ----> output : 4920
input : [290, 4920] ----> output : 2241
input : [290, 4920, 2241] ----> output : 287
input : [290, 4920, 2241, 287] ----> output : 257


In [18]:
for i in range(1,context_size+1):
    x = tokenizer.decode(enc_sample[:i])
    y = tokenizer.decode([enc_sample[i]])
    print(f"input : {x} ----> output : {y}")

input :  and ----> output :  established
input :  and established ----> output :  himself
input :  and established himself ----> output :  in
input :  and established himself in ----> output :  a


In [19]:
with open('the-verdict.txt','r',encoding = 'utf-8') as file:
    text = file.read()
    
enc_text = tokenizer.encode(text)
print(len(enc_text))

5145


In [20]:
import tiktoken
import torch
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self,text,tokenizer,maxlength,stride):
        self.input_ids = []
        self.target_ids = []
        
        self.token_ids = tokenizer.encode(text)
        
        for i in range(0,len(self.token_ids)-maxlength,stride):
            input_ids = self.token_ids[i:i+maxlength]
            target_ids = self.token_ids[i+1:i+maxlength+1]
            self.input_ids.append(input_ids)
            self.target_ids.append(target_ids)
            
    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, index):
        return torch.tensor(self.input_ids[index]),torch.tensor(self.target_ids[index])

In [21]:
def create_dataloader(text,batch_size=4,maxlength=256,
                      stride=128,drop_last=True,
                      num_workers = 0,
                      shuffle = True):
    
    tokenizer = tiktoken.get_encoding("gpt2")
    
    return DataLoader(
        dataset=GPTDatasetV1(text=text,tokenizer=tokenizer,maxlength=maxlength,stride=stride),
        batch_size=batch_size,
        num_workers=num_workers,
        drop_last=drop_last,
        shuffle = shuffle
    )

In [22]:
with open('the-verdict.txt','r',encoding = 'utf-8') as file:
    text = file.read()
    
train_dl = create_dataloader(text,batch_size=8,maxlength=4,stride=4,shuffle=False)

next(iter(train_dl))

#len(train_dl)

[tensor([[   40,   367,  2885,  1464],
         [ 1807,  3619,   402,   271],
         [10899,  2138,   257,  7026],
         [15632,   438,  2016,   257],
         [  922,  5891,  1576,   438],
         [  568,   340,   373,   645],
         [ 1049,  5975,   284,   502],
         [  284,  3285,   326,    11]]),
 tensor([[  367,  2885,  1464,  1807],
         [ 3619,   402,   271, 10899],
         [ 2138,   257,  7026, 15632],
         [  438,  2016,   257,   922],
         [ 5891,  1576,   438,   568],
         [  340,   373,   645,  1049],
         [ 5975,   284,   502,   284],
         [ 3285,   326,    11,   287]])]

In [23]:
next(iter(train_dl))

[tensor([[   40,   367,  2885,  1464],
         [ 1807,  3619,   402,   271],
         [10899,  2138,   257,  7026],
         [15632,   438,  2016,   257],
         [  922,  5891,  1576,   438],
         [  568,   340,   373,   645],
         [ 1049,  5975,   284,   502],
         [  284,  3285,   326,    11]]),
 tensor([[  367,  2885,  1464,  1807],
         [ 3619,   402,   271, 10899],
         [ 2138,   257,  7026, 15632],
         [  438,  2016,   257,   922],
         [ 5891,  1576,   438,   568],
         [  340,   373,   645,  1049],
         [ 5975,   284,   502,   284],
         [ 3285,   326,    11,   287]])]

In [24]:
vocab_size = 50257
output_dim = 256
token_embedding_layer = torch.nn.Embedding(vocab_size,output_dim)
input,target = next(iter(train_dl))
token_embeddings = token_embedding_layer(input)

In [25]:
print(token_embeddings.shape)

torch.Size([8, 4, 256])


In [26]:
context_length = 4
output_dim = 256
positional_embedding_layer = torch.nn.Embedding(context_length,output_dim)
positional_embeddings = positional_embedding_layer(torch.arange(context_length))

input_embeddings = token_embeddings + positional_embeddings

In [27]:
input_embeddings.shape

torch.Size([8, 4, 256])